In [2]:
#imports
import scipy as sp
import math
import numpy as np

import pprint  as pp
import matplotlib.pyplot as plt
from functools import partial
from ipywidgets import interact




In [12]:
#given params:
#E
c_Ena       = 45
c_Ek        =-80
c_El        =-59
#g
c_gna_bar    = 120
c_gk_bar     = 36
c_gl        = 0.3
#C (cap membrana)
c_C         = 1




In [13]:
#diffs
f_dVdt = lambda t , Cm , Ix , g , V , E : (Ix - g * (V-E) ) / Cm
f_dndt = lambda t , alpha_n , n , beta_n :  alpha_n*(1-n) - beta_n*n

#alpha|beta -> m -> g_i -> g -> E
f_alpha_m = lambda V : ( V + 45 )   /   ( 10 * (1 - np.exp (-(V+45)/10) ) )
f_alpha_n = lambda V : ( V + 60 )   /   ( 100 * (1 - np.exp (-(V+60)/10) )  )
f_beta_m  = lambda V : 4    *np.exp (   -(V+70) / 18 )   
f_beta_n  = lambda V : 0.125*np.exp (   -(V+70) / 80 )     

f_m     =lambda alpha_m , beta_m : alpha_m / (alpha_m + beta_m)

f_gna   = lambda gna_bar , m , n : gna_bar * m**3 * (1-n)
f_gk    = lambda gk_bar ,n : gk_bar *n**4
f_gl    = lambda : c_gl
f_g     = lambda gna , gk , gl: gna + gk + gl 
f_E     = lambda Ena, Ek, El, gna, gk , gl : np.average([Ena, Ek, El],weights=[gna,gk,gl])



0.3
